In [11]:
from collections import Counter
import numpy as np
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
from utils.hmatrStatistics import Hmatr

%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2
rpy2.robjects.numpy2ri.activate()

rssa = importr('Rssa')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Для простоты, пусть наш ряд задан как $x_n = sin(2\pi \omega n)$, причем $\begin{equation*} \omega = \begin{cases} \omega_1\ n \in [0, Q-1] \\ \omega_2\ n\in [Q, N-1] \end{cases} \end{equation*}$

Проведем аналитический анализ индекса неоднородности. Вспомним как он задается:
$$g(F^{(1)}; F^{(2)}) = \frac{\sum\limits_{l=1}^{K_2}\mathrm{dist}^2(X_l^{(2)}, \mathfrak{L}_r^{(1)})}{\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2} = \frac{\sum\limits_{l=1}^{K_2}\;(\|X_l^{(2)}\|^2 - \sum\limits_{i=1}^{r}\langle X_l^{(2)}, U_i^{(1)}\rangle^2)}{\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2} = 1 - \frac{\sum\limits_{l=1}^{K_2}\;\sum\limits_{i=1}^{r}\langle X_l^{(2)}, U_i^{(1)}\rangle^2}{\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2}$$


Начнем со знаменателя $\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2$, а точнее, с квадрата нормы $\|X_l^{(2)}\|^2$. Оценим его:

$$ \|X_l^{(2)}\|^2 = \sum\limits_{i=1}^{L}(X_{l_i}^{(2)}) = \int\limits_{0}^{L}X_{l_i}^{(2)} = \int\limits_{0}^{L}\sin^2{(2\pi\omega_2 y)}dy = \frac{L}{2} - \frac{\sin(4\pi L\omega_2)}{8\pi\omega_2} \approx \frac{L}{2} $$

Отсюда $\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2 \approx K_2\cdot\frac{L}{2}$.

Проверим равенство программно.

In [2]:
N = 700
w1 = 1/10
w2 = 1/5
C = 1
phi1 = 0
phi2 = 0
Q = 301  # 301 номер, значит разладка в ряде будет на 302й точке, если ряд задан с 0.
B = 100
T_ = 100
L = 50
r = 2
method = "svd"

In [3]:
def generate_series(omega, C=1, N=700, Q=301):
    w1, w2 = omega
    series = lambda n: C*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C*np.sin(2*np.pi*w2*n + phi1)
    return [series(i) for i in range(N)]

In [4]:
f = generate_series((w1, w2), N=N, Q=Q)
hm = Hmatr(f, B, T_, L, neig=r, svdMethod=method)

В формуле расстояния считаются между базовыми и тестовыми подрядами. В знаменателе $X_l^{(2)}$ -  столбцы траекторной матрицы тестового подряда (в формуле (2.1) ряда $F^{(2)}$). Длина таких рядов равна $T$, а длина окна равна $L$. Отсюда $K_2 = T - L + 1$

In [21]:
K2 = T_ - L + 1
K2

51

In [22]:
K2 * L/2

1275.0

Проверим полученную аналитически оценку по эмпирически.

In [19]:
sum(hm.STATISTICS['norm'])/len(hm.STATISTICS['norm'])

1275.0

Получили то же самое число, правда в среднем. Посмотрем на значения, которые встречались:

In [12]:
Counter(hm.STATISTICS['norm'])

Counter({1275.0: 560, 1275.5590169943753: 20, 1274.4409830056247: 20})

Как мы видим, значения не сильно отклоняются от среднего. Откуда можно заключить, что хвост $\frac{\sin(4\pi L\omega_2)}{8\pi\omega_2}$, отнимаемый от $\frac{L}{2}$ ассимптотически не влияет на оценку $\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2$ и при достаточно больших $L$ мы имеем равенство: $\sum\limits_{l=1}^{K_2}\|X_l^{(2)}\|^2 = K_2\cdot\frac{L}{2}$.

Для подтверждения нашего предположения, проведем еще несколько экспериментов.

---

1

In [20]:
L = 990
N = 1700
Q = 801  # 301 номер, значит разладка в ряде будет на 302й точке, если ряд задан с 0.
B = 1000
T_ = 1000
f = generate_series((w1, w2), N=N, Q=Q)
hm = Hmatr(f, B, T_, L, neig=r, svdMethod=method)
print(sum(hm.STATISTICS['norm'])/len(hm.STATISTICS['norm']))
print(Counter(hm.STATISTICS['norm']))

5445.0
Counter({5445.0: 394, 5444.440983005625: 132, 5445.559016994375: 131, 5445.000000000002: 10, 5445.000000000004: 7, 5444.999999999993: 6, 5445.559016994377: 3, 5445.000000000001: 3, 5445.559016994368: 3, 5444.440983005626: 2, 5444.440983005627: 2, 5445.559016994379: 2, 5444.440983005628: 2, 5444.440983005617: 2, 5445.559016994376: 1})


In [21]:
K2 = T_ - L + 1
K2 * L/2

5445.0

---

2

In [24]:
L = 400
N = 1700
Q = 301  # 301 номер, значит разладка в ряде будет на 302й точке, если ряд задан с 0.
B = 1000
T_ = 1000
f = generate_series((w1, w2), N=N, Q=Q)
hm = Hmatr(f, B, T_, L, neig=r, svdMethod=method)
print(sum(hm.STATISTICS['norm'])/len(hm.STATISTICS['norm']))
print(Counter(hm.STATISTICS['norm']))

120199.90416851516
Counter({120200.0: 580, 120199.44098300562: 120})


In [25]:
K2 = T_ - L + 1
K2 * L/2

120200.0

---

Получили примерно те же самые значения и чем больше L, тем меньше влияет $\frac{\sin(4\pi L\omega_2)}{8\pi\omega_2}$.

Можно также заметить, что при сравнительно небольших $L$ и больших $K_2$ влияние слагаемого в разности может увеличится и стать существенным.